In [42]:
import numpy as np

class Layer:
    def __init__(self, input_left_mt: np.ndarray, input_right_mt: np.ndarray, var_index):
        self.input_left_mt = input_left_mt
        self.input_right_mt = input_right_mt
        self.var_index = var_index
        self.downer_block, self.upper_block, self.zero_ineq = self.distinguishing()

    @property
    def total_right(self):
        right_mt = self.input_right_mt - self.input_left_mt
        right_mt[:, self.var_index] = 0
        return right_mt
    
    @property
    def total_left(self):
        left_mt = self.input_left_mt - self.input_right_mt
        left_mt[:,[i for i in range(left_mt.shape[1]) if i != self.var_index]] = 0
        return left_mt

    def distinguishing(self):
        possitive_ineq_index = self.total_left[:,self.var_index] > 0
        negative_ineq_index = self.total_left[:,self.var_index] < 0
        zero_ineq_index = self.total_left[:,self.var_index] == 0

        upper_block = self.total_right[possitive_ineq_index] / self.total_left[possitive_ineq_index, self.var_index][:, None]
        downer_block = self.total_right[negative_ineq_index] / self.total_left[negative_ineq_index, self.var_index][:, None]

        return downer_block, upper_block, -self.total_right[zero_ineq_index]
    
    def __repr__(self):
        return f"Downer: \n{self.downer_block}\nUpper: \n{self.upper_block}\nZero: \n{self.zero_ineq}"



class Scheme:
    def __init__(self, input_matrix: np.ndarray):
        self.input_matrix = input_matrix
        self.nvar = input_matrix.shape[1]
        self.nequal = input_matrix.shape[0]
    
    @property
    def layers(self):
        layer_stack = []
        input_mt = self.input_matrix
        for lay_index in range(self.nvar):
            lay = Layer(input_left_mt=input_mt, input_right_mt=np.zeros(input_mt.shape), var_index=lay_index)

            layer_stack.append(lay)


            downer_block = lay.downer_block
            upper_block = lay.upper_block
            zero_ineq = lay.zero_ineq

            new_ineqs = np.vstack(((
                downer_block[:, None, :] - upper_block[None, :, :]).reshape(-1, self.nvar),
                zero_ineq
            ))
            
            input_mt = new_ineqs
        
        return layer_stack

        


input_matrix = np.array([
    [1,2,-3],        #   a + 2b - 3c >= 0 
    [-2,0,1],        # -2a + 0b +  c >= 0
    [2,-1,0]         #  2a -  b + 0c >= 0
])

scheme = Scheme(input_matrix)
scheme.layers

[Downer: 
 [[-0.  -0.   0.5]]
 Upper: 
 [[ 0.  -2.   3. ]
  [ 0.   0.5  0. ]]
 Zero: 
 [],
 Downer: 
 [[-0. -0.  1.]]
 Upper: 
 [[0.   0.   1.25]]
 Zero: 
 [],
 Downer: 
 [[-0. -0. -0.]]
 Upper: 
 []
 Zero: 
 []]